This is a standalone notebook to solve a jigsaw puzzle

# Import dependencies

In [ ]:
%pip install matplotlib opencv-python scipy tqdm;

In [ ]:
import matplotlib.pyplot as plt
import glob
import numpy as np
import scipy.stats
import cv2
import math
import random
import itertools
import json
import lzma
import tqdm.notebook as tqdm
from functools import cache
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
from collections import Counter

# Add utilities

In [ ]:
def imshow(img, title=None):
    plt.title(title)
    plt.imshow(img[0:1680, 0:1700])
    plt.show()

class Item():
    def __init__(self, **kwargs):
        self.update(**kwargs)

    def update(self, **kwargs):
        self.__dict__.update(kwargs)

class LoopingList(list):
    def __getitem__(self, i):
        if isinstance(i, int):
            return super().__getitem__(i % len(self))
        else:
            return super().__getitem__(i)

def plot_contour(contour, **kwargs):
    plt.plot(contour[:, :, 0], contour[:, :, 1], **kwargs)

def fill_contour(contour, **kwargs):
    plt.fill(contour[:, :, 0], contour[:, :, 1], **kwargs)

def sub_contour(c, idx0, idx1):
    if idx1 > idx0:
        return c[idx0:idx1]
    else:
        return np.concatenate([c[idx0:], c[:idx1]])

# Detect pieces

In [ ]:
images = []
pieces = []
for filename in tqdm.tqdm(sorted(glob.glob("scans/kaggle/*.jpg"))):
    img_original = cv2.imread(filename)
    h, w = img_original.shape[:2]
    img_gray = cv2.cvtColor(img_original, cv2.COLOR_BGR2GRAY)
    _, img_binary = cv2.threshold(img_gray, 30, 255, cv2.THRESH_BINARY)
    # no blur, it causes more harm than good for these images
    raw_contours, _ = cv2.findContours(img_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    raw_contours = [c for c in raw_contours if cv2.contourArea(c) > 100e3]
    # draw fully filled contours
    img_contours = np.zeros((h, w), dtype=np.uint8)
    for contour in raw_contours:
        cv2.drawContours(img_contours, [contour], 0, (255, 255, 255), -1)
    # remove small connected dirt
    img_corrected = cv2.morphologyEx(img_contours, cv2.MORPH_OPEN, np.ones((9, 9), dtype=np.uint8))
    # get the clean contour
    contours, _ = cv2.findContours(img_corrected, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contours = [c for c in contours if cv2.contourArea(c) > 100e3]
    img_masked = img_gray & img_corrected
    img_pieces = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        piece = Item(
            contour=contour - (x, y),
            img=img_masked[y:y+h, x:x+w],
            filename=filename,
            filepos=(x, y)
        )
        img_pieces.append(piece)
    
    pieces.extend(img_pieces)
    
    image = Item(
        filename=filename,
        img_gray=img_gray,
        pieces=img_pieces,
    )
    images.append(image)

for id, piece in enumerate(pieces):
    piece.update(id=id)

print(f"Detected {len(pieces)} pieces")

In [ ]:
# lzma img gray: 21s, 11MB
# np.savez img gray: 0.1s, 35MB
# np.savez_compressed img gray: 1.6s, 13MB
# for image in images:
#     print(image.filename)
#     np.savez_compressed(image.filename + ".np", image.img_gray)
# dump_pieces(pieces)
# print(pieces[0].__dict__.keys())

# json.dumps(images[0].pieces[0])
# for image in images:
#     plt.title(image.filename)
#     plt.imshow(image.img_gray)
#     for piece in image.pieces[1:]:
#         fill_contour(piece.contour + piece.filepos, c="red")
#     plt.show()

In [ ]:
# show the smallest and biggest pieces by area
pieces.sort(key= lambda piece: cv2.contourArea(piece.contour))

for piece in pieces[:1] + pieces[-1:]:
    img = np.copy(piece.img)
    plt.title(f"Area={int(cv2.contourArea(piece.contour))}")
    plt.imshow(img)
    plot_contour(piece.contour, c="red", ls="--")
    plt.show()

# Detect piece corners

## Find corners via peak distance from center

In [ ]:
for piece in tqdm.tqdm(pieces):
    (cx, cy), r = cv2.minEnclosingCircle(piece.contour)
    centered_contour = piece.contour - np.array([cx, cy])
    # ensure peaks are not at start or end of the distances array
    distances = np.sum(centered_contour**2, axis=2)[:, 0]
    distance_offset = np.argmin(distances)
    distances = np.concatenate([distances[distance_offset:], distances[:distance_offset]])

    # find peak distances
    peak_indices = [(distance_idx + distance_offset) % len(distances) for distance_idx in find_peaks(distances, prominence=10000)[0]]    
    peak_indices.sort()
    piece.update(peak_indices=LoopingList(peak_indices))

In [ ]:
# Show the pieces having the smallest / highest number of peak indices
pieces.sort(key= lambda piece: len(piece.peak_indices))

for piece in pieces[:1] + pieces[-1:]:
    img = np.copy(piece.img)
    plt.title(f"Number of peaks={len(piece.peak_indices)}")
    for idx in piece.peak_indices:
        cv2.circle(img, piece.contour[idx, 0], 10, (255, 0, 0), 3)
    plt.imshow(img)
    plot_contour(piece.contour, c="red", ls="--")
    plt.show()

## Save/Restore snapshot

In [ ]:
assert len(pieces) == 1998

np.savez_compressed("kaggle_piece.contour.npz", **dict([(str(idx), piece.contour) for idx, piece in enumerate(pieces)]))
np.savez_compressed("kaggle_piece.img.npz", **dict([(str(idx), piece.img) for idx, piece in enumerate(pieces)]))

jpieces = []
for idx, piece in enumerate(pieces):
    jpieces.append(dict(
        id=piece.id,
        filename=piece.filename,
        filepos=[int(x) for x in piece.filepos],
        peak_indices=[int(x) for x in piece.peak_indices],
    ))

with open("kaggle_piece.json", "w") as f:
    json.dump(jpieces, f)

print(f"Dumped {len(jpieces)} pieces to files")

In [ ]:
pieces = []

piece_contours = np.load("kaggle_piece.contour.npz")
piece_imgs = np.load("kaggle_piece.img.npz")

with open("kaggle_piece.json", "r") as f:
    jpieces = json.load(f)
    for idx, jpiece in enumerate(jpieces):
        contour = piece_contours[str(idx)]
        img = piece_imgs[str(idx)]
        piece = Item(
            id=jpiece['id'],
            filename=jpiece['filename'],
            filepos=tuple(jpiece['filepos']),
            peak_indices=[np.int64(x) for x in jpiece['peak_indices']],
            img=img,
            contour=contour,
        )
        pieces.append(piece)

print(f"Loaded {len(pieces)} pieces from files")

## Filter corners by angle geometry

In [ ]:
for piece in tqdm.tqdm(pieces):
    def is_right_angle(idx):
        N = len(piece.contour)
        angle_contour = np.concatenate([piece.contour[idx:], piece.contour[:idx]]) - piece.contour[idx]
        contour0 = angle_contour[math.ceil(N*0.005):int(N*0.04)]
        contour1 = angle_contour[-int(N*0.04):-math.ceil(N*0.005)]
        angles0 = np.arctan2(contour0[:, 0, 1], contour0[:, 0, 0]) % (2 * np.pi)
        angles1 = np.arctan2(contour1[:, 0, 1], contour1[:, 0, 0]) % (2 * np.pi)
        mean0 = scipy.stats.circmean(angles0)
        mean1 = scipy.stats.circmean(angles1)
        std0 = scipy.stats.circstd(angles0)
        std1 = scipy.stats.circstd(angles1)
        angle = (mean0 - mean1) % np.pi
        return -0.3 < angle - np.pi/2 < 0.3 and std0 < 0.1 and std1 < 0.1

    # find peaks that look like right angles
    angle_indices = [idx for idx in piece.peak_indices if is_right_angle(idx)]
    piece.update(angle_indices=LoopingList(angle_indices))

In [ ]:
# Show the pieces having the smallest / highest number of angle indices
pieces.sort(key= lambda piece: len(piece.angle_indices))

for piece in pieces[:1] + pieces[-1:]:
    img = np.copy(piece.img)
    plt.title(f"Number of peaks={len(piece.angle_indices)}")
    for idx in piece.angle_indices:
        cv2.circle(img, piece.contour[idx, 0], 10, (255, 0, 0), 3)
    plt.imshow(img)
    plot_contour(piece.contour, c="red", ls="--")
    plt.show()

## Filter corners by rectangle geometry

In [ ]:
for piece in tqdm.tqdm(pieces):
    def compute_rectangle_error(indices):
            # get coordinates of corners
            corners = LoopingList(np.take(piece.contour, sorted(list(indices)), axis=0)[:, 0, :])
            # compute the side lengths and diagonal lengths
            lengths = [math.sqrt(np.sum((corners[i0] - corners[i1])**2)) for i0, i1 in [(0, 1), (1, 2), (2, 3), (3, 0), (0, 2), (1, 3)]]
            def f_error(a, b):
                return abs(b - a) / (a + b)
            return sum([f_error(lengths[i], lengths[j]) for i, j in [(0, 2), (1, 3), (4, 5)]])

    # form a good rectangle with corner indices and missing indices
    rectangles = []  # list of (score, [indices])
    for indices in itertools.combinations(piece.peak_indices, 4):
        if set(indices).issuperset(piece.angle_indices) or len(piece.angle_indices) > 4:
            error = compute_rectangle_error(indices)
            rectangles.append((error, indices))

    if len(rectangles) >= 1:
        error, indices = sorted(rectangles)[0]
    else:
        error, indices = [100, piece.angle_indices]
    piece.update(rectangle_error=error)
    piece.update(corner_indices=LoopingList(indices))


In [ ]:
# Show the pieces having the best / worst rectangle
pieces.sort(key= lambda piece: piece.rectangle_error)

for piece in pieces[:1] + pieces[-1:]:
    img = np.copy(piece.img)
    plt.title(f"Rectangle error={piece.rectangle_error}")
    for idx in piece.corner_indices:
        cv2.circle(img, piece.contour[idx, 0], 10, (255, 0, 0), 3)
    plt.imshow(img)
    plot_contour(piece.contour, c="red", ls="--")
    plt.show()

# Analyze edges

## Extract edges & detect sign

In [ ]:
for piece in pieces:
    edges = LoopingList()
    contour = piece.contour.astype(np.float64)  # convert to float for rotation
    for quarter in range(4):
        idx0 = piece.corner_indices[quarter]
        idx1 = piece.corner_indices[quarter+1]
        p0 = contour[idx0][0]
        p1 = contour[idx1][0]
        # normalize the contour: first point at (0, 0), last point at (X, 0)
        dx, dy = p1 - p0
        angle_radians = math.atan2(dy, dx)
        matrix = cv2.getRotationMatrix2D(p0, math.degrees(angle_radians), 1)
        normalized_piece_contour = cv2.transform(contour, matrix) - p0
        normalized_edge_contour = sub_contour(normalized_piece_contour, idx0, idx1 + 1)

        # compute the sign of the edge
        heights = normalized_edge_contour[:, 0, 1]
        if np.max(np.abs(heights)) > 20:
            sign = 1 if np.max(heights) > - np.min(heights) else -1
        else:
            sign = 0

        edge = Item(
            idx0=idx0,
            idx1=idx1,
            normalized_piece_contour=normalized_piece_contour,
            sign=sign,
        )
        edges.append(edge)
    
    for idx, edge in enumerate(edges):
        edge.update(
            prev=edges[idx-1],
            next=edges[idx+1]
        )
    
    piece.update(
        edges=edges,
        nb_flats=len([edge for edge in edges if edge.sign == 0])
    )

print("edge sign:", Counter([edge.sign for piece in pieces for edge in piece.edges]))
print("nb of flats:", Counter([piece.nb_flats for piece in pieces]))

In [ ]:
# Show the pieces having the smallest / highest number of flats
pieces.sort(key= lambda piece: piece.nb_flats)

sign2color = {-1: "red", 0: "green", 1: "yellow"}

for piece in pieces[:1] + pieces[-1:]:
    img = np.copy(piece.img)
    plt.title(f"Nb of flats={piece.nb_flats}")
    for edge in piece.edges:
        # sign = random.choice([-1, 0, 1])
        # print(sign)
        plot_contour(sub_contour(piece.contour, edge.idx0, edge.idx1), c=sign2color[edge.sign])
    plt.imshow(img)
    plt.show()

## Compute puzzle size

In [ ]:
def compute_size(area, perimeter):
    # perimeter = 2 * (H+W)
    # area = H*W
    # H**2 - perimeter/2 * H + area = 0
    a = 1
    b = -perimeter/2
    c = area
    delta = b**2 - 4*a*c
    h = int((-b - math.sqrt(delta)) / (2*a))
    w = int((-b + math.sqrt(delta)) / (2*a))
    return (min(h, w), max(h, w))

solution = Item()

nb_flats = Counter([piece.nb_flats for piece in pieces])
assert nb_flats[2] == 4
area = len(pieces) + 2   # there are 2 missing pieces in the kaggle dataset
perimeter = nb_flats[1] + 2*nb_flats[2]
w, h = compute_size(area, perimeter)
print(f"Size of puzzle grid: {w} x {h}")
assert w * h == area
assert 2 * (w + h) == perimeter

solution.update(grid_size = (w, h))

## Sample edges

In [ ]:
NB_SAMPLES = 19

for piece in tqdm.tqdm(pieces):
    for edge in piece.edges:
        # compute the distance from the first point, this is not exactly edge.arc_length
        edge_contour = sub_contour(edge.normalized_piece_contour, edge.idx0, edge.idx1)
        deltas = edge_contour[1:] - edge_contour[:-1]
        distances = np.cumsum(np.sqrt(np.sum(deltas**2, axis=2)))
        distance = distances[-1] / (NB_SAMPLES - 1)  # distance between 2 sample points
        # get N equidistant points
        sample_indices = (np.array([np.argmax(distances >= i*distance - 0.0001) for i in range(NB_SAMPLES)]) + edge.idx0) % len(piece.contour)

        edge.update(
            sample_indices=sample_indices,
        )

In [ ]:
pieces.sort(key= lambda piece: piece.id)

for piece in pieces[:1]:
    plt.axis('equal')
    plt.title(f"id {piece.id}")
    plt.imshow(piece.img)
    plot_contour(piece.contour, c="yellow", ls=':')
    for edge, marker in zip(piece.edges, 'x^+v'):
        plot_contour(piece.contour[edge.sample_indices], marker=marker, ls='', c="red")
    plt.axvline(x=0, c="gray", ls=":")
    plt.axhline(y=0, c="gray", ls=":")
    plt.show()

# Compute the border

In [ ]:
flat_pieces = []

for piece in pieces:
    if piece.nb_flats > 0:
        flat_edges = [edge for edge in piece.edges if edge.sign == 0]
        piece.update(
            first_flat = flat_edges[0],
            last_flat = flat_edges[-1],
            before_flat = flat_edges[0].prev,
            after_flat = flat_edges[-1].next,
        )
        flat_pieces.append(piece)

def plot_border_match(piece0, piece1):
    plt.axis('equal')
    contour0 = piece0.last_flat.normalized_piece_contour
    plot_contour(contour0[piece0.after_flat.sample_indices], ls='', marker='+', c='red')
    offset = contour0[piece0.last_flat.idx1, 0]
    contour1 = piece1.first_flat.normalized_piece_contour
    plot_contour(contour1[piece1.before_flat.sample_indices] + offset, ls='', marker='x', c='blue')
    plt.show()

def plot_border_line(ordered_border):
    offset = np.array([0., 0.])
    plt.axis('equal')
    for idx, piece in enumerate(ordered_border):
        contour = (piece.last_flat if idx==0 else piece.first_flat).normalized_piece_contour
        if idx == 0:
            # plot_contour(contour + offset)
            plot_contour(contour[piece.after_flat.sample_indices] + offset, ls='', marker='+', c='red')
            offset += contour[piece.last_flat.idx1, 0]
        else:
            plot_contour(contour[piece.before_flat.sample_indices] + offset, ls='', marker='x', c='red')
            # plot_contour(contour + offset)
            plot_contour(contour[piece.after_flat.sample_indices] + offset, ls='', marker='+', c='red')
    plt.show()

In [ ]:
PAD = (300, 700)

# matrix = cv2.getRotationMatrix2D(p0, math.degrees(angle_radians), 1)
# normalized_piece_contour = cv2.transform(contour, matrix) - p0

# img = np.pad(p0.img, PAD)
# plt.imshow(img)
# plot_contour(sub_contour(p0.contour, e0.idx0, e0.idx1) + (PAD, PAD), c='red')
# plt.show()

# Thick binary mask following the edge
# img_mask = np.zeros_like(img)
# cv2.polylines(img_mask, [sub_contour(p0.contour, e0.idx0+10, e0.idx1-10)], False, 255, 20)
# plt.imshow(img_mask)
# plt.show()

# Binary mask of the first piece
# img = np.zeros((1000, 1000), dtype=np.uint8)
# cv2.fillPoly(img, [contour0.astype(int) + PAD], 255)
# plt.imshow(img)
# plt.show()
# 
# piece1 = pieces[1579]
# flat1 = piece1.last_flat
# contour1 = flat1.normalized_piece_contour
# plt.axis('equal')
# plot_contour(contour0)
# plot_contour(contour0[[flat0.idx1]], ls='', marker='x', c='red')
# 
# point0 = contour0[flat0.idx1]
# plot_contour(contour1 + point0)
# plt.show()
# 
# # img = np.pad(p1.img, PAD)
# 
# # Binary mask of the second piece
# img_p1 = np.zeros_like(img)
# img = np.zeros((1000, 1000), dtype=np.uint8)
# cv2.fillPoly(img, [contour0.astype(int)], 255)
# 
# plt.imshow(img)
# plt.show()
# 
# plt.imshow(img_p1)
# plt.show()
# 
# img_xor = cv2.bitwise_xor(img_p1, img_p0)
# plt.imshow(img_xor)
# plt.show()

# cv2.polylines(img_mask, [sub_contour(p0.contour, e0.idx0+10, e0.idx1-10)], False, 255, 20)
# plt.imshow(img_mask)
# plt.show()
# 
# cv2.bitwise_and(img_mask, img)

# 
# plt.imshow(img)
# plot_contour(sub_contour(p1.contour, e1.idx0, e1.idx1) + (PAD, PAD), c='red')
# plt.show()

def transform_contour(contour, idx, xy, degrees):
    matrix = cv2.getRotationMatrix2D(contour[idx][0], degrees, 1)
    return cv2.transform(contour, matrix) + xy - contour[idx][0]

piece0 = pieces[7]
flat0 = piece0.first_flat
contour0 = flat0.normalized_piece_contour

plot_contour(contour0)
plot_contour(contour0[[flat0.idx1]], ls='', marker='x', c='red')

contourX = transform_contour(contour0, flat0.idx1, (200, -200), 12)
plot_contour(contourX)
plot_contour(contourX[[flat0.idx1]], ls='', marker='x', c='red')



Match ideas:
* sign (male/female)
* edge arc length
* edge straight length
* xor pieces
* distance of sample points

In [ ]:
after_flat_features = {}  # key=piece, value=features
before_flat_features = {}

for piece in flat_pieces:
    for edge in piece.edges:
        if edge.sign == 0 and edge.prev.sign != 0:
            before_flat_features[piece] = edge.normalized_piece_contour[edge.prev.sample_indices][::-1]

        if edge.sign == 0 and edge.next.sign != 0:
            after_flat_features[piece] = edge.normalized_piece_contour[edge.next.sample_indices]

def piece_after_flat(piece0):
    features0 = after_flat_features[piece0]
    results = []
    for piece1, features1 in before_flat_features.items():
        diff = features1 - features0
        offset = np.mean(diff, axis=0)
        error = np.sum((diff - offset)**2)
        results.append((error, piece1))
    results.sort()
    print("after", piece0.id, [(int(error), piece.id) for error, piece in results[:3]])
    return results[0][1]

def piece_before_flat(piece0):
    features0 = before_flat_features[piece0]
    results = []
    for piece1, features1 in after_flat_features.items():
        diff = features1 - features0
        offset = np.mean(diff, axis=0)
        error = np.sum((diff - offset)**2)
        results.append((error, piece1))
    results.sort()
    print("before", piece0.id, [(int(error), piece.id) for error, piece in results[:3]])
    return results[0][1]

for piece in flat_pieces[:10]:
    best_next_piece = piece_after_flat(piece)
    best_prev_next_piece = piece_before_flat(best_next_piece)
    if piece != best_prev_next_piece:
        print(f"Mismatch for pieces: {piece.id}-{best_next_piece.id} or {best_prev_next_piece.id}-{best_next_piece.id}")
        # plt.imshow(piece.img)
        # plt.show()
        # plt.imshow(best_prev_next_piece.img)
        # plt.show()
        plot_border_match(piece, best_next_piece)
        plot_border_match(best_prev_next_piece, best_next_piece)

# # first_piece = [piece for piece in flat_pieces if piece.nb_flats == 2][1]
# last_piece = flat_pieces[4]
# ordered_border = [last_piece]
# while True:
#     best_next_piece = piece_after_flat(last_piece)
#     best_prev_next_piece = piece_before_flat(best_next_piece)
#     if last_piece != best_prev_next_piece:
#         print(f"This is maybe not the best piece", last_piece.id, best_prev_next_piece.id, best_next_piece.id)
#     last_piece = best_next_piece
#     ordered_border.append(last_piece)
#     if last_piece in ordered_border[1:]:
#         break
# 
# # print(f"Used pieces: {len(used_pieces)}/{nb_flats[1] + nb_flats[2]}")
# print(' '.join([str(piece.id) + ('*' if piece.nb_flats == 2 else '') for piece in ordered_border]))
# print("Computed border pieces:", ' '.join([piece.name for piece in ordered_border]))
# assert len(used_pieces) == nb_flats[1] + nb_flats[2]
# assert ordered_border[-1] == ordered_border[0]  # loop on the first piece
# ordered_border = ordered_border[:-1]  # remove the repeated first piece
# h, w = solution.grid_size
# if ordered_border[h-1].nb_flats == 1:
#     h, w = w, h
#     solution.grid_size = w, h
# assert [idx for idx, piece in enumerate(ordered_border) if piece.nb_flats == 2] == [0, h-1, h+w-2, 2*h+w-3]


# Place the border

In [ ]:
PAD = 30

solution.update(
    grid={} # key=(i, j), value=piece
)

def last_flat_edge(piece):
    return [edge for edge in piece.edges if edge.sign == 0 and edge.next.sign != 0][0]

def first_flat_edge(piece):
    return [edge for edge in piece.edges if edge.sign == 0 and edge.prev.sign != 0][0]

def place_piece(ij, piece, xy, idx, top_edge_idx, flat_edge, degrees):
    i, j = ij
    solution.grid[(i, j)] = piece
    dx, dy = piece.contour[flat_edge.idx1][0] - piece.contour[flat_edge.idx0][0]
    matrix = cv2.getRotationMatrix2D(piece.contour[idx][0], degrees + math.degrees(math.atan2(dy, dx)), 1)
    piece.contour = cv2.transform(piece.contour, matrix) + xy - piece.contour[idx][0]
    piece.edges = piece.edges[top_edge_idx:] + piece.edges[:top_edge_idx]

def place_border():
    pos = np.array([0, 0])
    xy = (0, 0)
    it_pieces = iter(ordered_border)
    for quarter, dpos in zip([0, 3, 2, 1, 0], [(0, 1), (1, 0), (0, -1), (-1, 0), (0, 1)]): 
        for piece in it_pieces:
            flat_edge = first_flat_edge(piece)  # this edge is aligned on previous piece
            top_edge_idx = piece.edges.index(flat_edge) + quarter
            if np.all(pos == 0):
                idx = flat_edge.idx0  # reference (xy) point is the corner
            else:
                idx = flat_edge.idx1
            place_piece(tuple(pos), piece, xy, idx, top_edge_idx, flat_edge, 90 * quarter)
            pos += dpos
            xy = piece.contour[last_flat_edge(piece).idx0]
            if piece.nb_flats == 2:
                break

place_border()

for piece in solution.grid.values():
   plt.axis('equal')
   plt.axvline(x=0, c="gray", ls=":")
   plt.axhline(y=0, c="gray", ls=":")
   plot_contour(piece.contour)

# Add inner pieces

In [ ]:
bottom_left_features = {}  # key=(piece, edge_idx), value=features

for piece in pieces:
    for idx, edge in enumerate(piece.edges):
        contour = edge.normalized_piece_contour
        sample_indices = np.concatenate([edge.next.sample_indices[::-1], edge.prev.prev.sample_indices[::-1]])
        bottom_left_features[(piece, idx)] = contour[sample_indices]
        # plot_contour(edge.normalized_piece_contour)
        # plot_contour(contour[sample_indices][:5], ls='', marker='o')

def edge_degrees(piece, edge_idx):
    contour = piece.contour
    edge = piece.edges[edge_idx]
    dx, dy = contour[edge.idx1][0] - contour[edge.idx0][0]
    return math.degrees(math.atan2(dy, dx))

def piece_bottom_left(piece_left, piece_bottom):
    points_left = piece_left.contour[piece_left.edges[3].sample_indices]
    # plot_contour(piece_left.contour)
    # plot_contour(points_left, ls='', marker='o')

    points_bottom = piece_bottom.contour[piece_bottom.edges[0].sample_indices]
    # plot_contour(piece_bottom.contour)
    # plot_contour(points_bottom, ls='', marker='o')

    angle_degrees = edge_degrees(piece_bottom, 0)
    matrix = cv2.getRotationMatrix2D(points_left[-1][0], angle_degrees, 1)
    features0 = cv2.transform(np.concatenate([points_left, points_bottom]), matrix)
    plot_contour(features0, ls='', marker='o')

    results = []
    for (piece1, edge1_idx), features1 in bottom_left_features.items():
        diff = features1 - features0
        offset = np.mean(diff, axis=0)
        score = np.sum((diff - offset)**2)
        results.append((score, piece1, edge1_idx))
        if score < 1000:
            plot_contour(features1 - offset)
    results.sort()
    print([r[0] for r in results[:5]])
    return min(results)[1:]

def place_at(i, j):
    piece_left = solution.grid[(i-1, j)]
    piece_bottom = solution.grid[(i, j+1)]
    piece, top_edge_idx = piece_bottom_left(solution.grid[(i-1, j)], solution.grid[(i, j+1)])
    bottom_edge = piece.edges[top_edge_idx+2]
    ref_point = piece_bottom.contour[piece_bottom.edges[0].idx0][0]
    place_piece((i, j), piece, ref_point, bottom_edge.idx1, top_edge_idx, bottom_edge, 180 - edge_degrees(piece_bottom, 0))

    # plot_contour(piece_left.contour)
    # plot_contour(piece_left.contour[piece_left.edges[3].sample_indices], ls='', marker='x')
    # plot_contour(piece_bottom.contour)
    # plot_contour(piece_bottom.contour[piece_bottom.edges[0].sample_indices], ls='', marker='x')
    # plot_contour(piece.contour)

# piece_at(1, 10)
# for i in range(2):
#     place_at(i+1, 10)
place_at(1, 10)

#for piece in solution.grid.values():
#   plot_contour(piece.contour)

#plot_contour(pieces[5].edges[2].normalized_piece_contour)
#plot_contour(pieces[5].edges[2].normalized_piece_contour[pieces[5].edges[2].sample_indices], ls='', marker='o')
# plot_contour(solution.grid[(i, j)].contour)
# plot_contour(solution.grid[(i, j)].contour)
# plot_contour(solution.grid[(i, j)].contour)
# plot_contour(solution.grid[(i, j)].contour)